Add github source as a path to be accessed by kaggle

In [1]:
import os

Specifying which model to train & evaluate

In [2]:
model_to_train = 'wgan-gp'
# model_to_train = 'wgan-gp'


Configurations specific to run

In [3]:
config = {
    # environment
    'environment': 'local',
    'local_results_directory': './results',
    'experiment_name': 'v1.6',
    'data_directory': './data/faces_reduced',
    'evaluation': True,
    'num_workers': 0,

    # network
    'noise_size': 100,
    'noise_type': 'normal', # uniform / normal
    'discriminator_feature_map_depth': 64,
    'generator_feature_map_depth': 64,

    # training
    'save_checkpoint_every': 10,
    'save_image_every': 10,
    'save_metrics_every': 10,
    'batch_size': 64,
    'epochs': 20,
    'discriminator_lr': 0.002,
    'discriminator_betas': (0.5, 0.999),
    'generator_lr': 0.002,
    'generator_betas': (0.5, 0.999),
    'true_label_value': 1,
    'fake_label_value': 0,

    # model
    'model_name': model_to_train,

    # model specific settings
    # wgan settings
    'weight_clip': 0.1,

    # wgan-gp settings
    'critic_iterations': 5,
    'lambda_gp': 10,
    'wgan_gp_lr': 1e-4,
    'wgan_gp_betas': (0.0, 0.9)

}

# create paths
if not os.path.isdir(config['local_results_directory']):
    os.mkdir(config['local_results_directory'])


Executing based on the configurations

In [4]:
import torch as th
import torchvision
from torch.utils.data import DataLoader

from models import Generator, Discriminator
import models_wgan.wgan_gp as wgan_gp
from utils import weights_init
from experiments import Experiment

# create device
device = th.device('cuda' if th.cuda.is_available() else 'cpu')

# create dataset
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])
dataset = torchvision.datasets.ImageFolder(config['data_directory'], transform=transform)
dataloader = DataLoader(dataset, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_workers'])

if config['model_name'] == 'dcgan':
    # create networks
    generator = Generator(config['noise_size'],config['generator_feature_map_depth']).to(device)
    discriminator = Discriminator(config['discriminator_feature_map_depth']).to(device)
    generator.apply(weights_init)
    discriminator.apply(weights_init)

    # create optimizers
    discriminator_optimizer = th.optim.Adam(discriminator.parameters(), lr=config['discriminator_lr'], betas=config['discriminator_betas'])
    generator_optimizer = th.optim.Adam(generator.parameters(), lr=config['generator_lr'], betas=config['generator_betas'])

    # create loss
    criterion = th.nn.BCELoss()
    # create experiment
    experiment = Experiment(config, generator, discriminator, generator_optimizer, discriminator_optimizer, criterion, dataloader)
    print('Training dcgan')
    experiment.train()


# elif config['model_name'] == 'dcgan-data-aug':
#
# elif config['model_name'] == 'wgan':
#
elif config['model_name'] == 'wgan-gp':
    # create networks
    generator = wgan_gp.Generator(config["noise_size"], config["generator_feature_map_depth"]).to(device)
    critic = wgan_gp.Critic(config["discriminator_feature_map_depth"]).to(device)
    generator.apply(weights_init)
    critic.apply(weights_init)

    # create optimizers
    # Optimizer (WGAN uses RMSprop, WGAN-GP uses Adam)
    critic_optimizer = th.optim.Adam(critic.parameters(), lr=config["wgan_gp_lr"], betas=config["wgan_gp_betas"])
    generator_optimizer = th.optim.Adam(generator.parameters(), lr=config["wgan_gp_lr"], betas=config["wgan_gp_betas"])

    generator.train()
    critic.train()
    criterion = None

    experiment = wgan_gp.Training(generator, critic, generator_optimizer, critic_optimizer, device, dataloader, config)
    experiment.train()

# elif config['model_name'] == 'wgan-gp-data-aug':




/Users/fransdeboer/Projects/CS4245/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


EPOCH:  0
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.])
tensor([-0.1889,  0.8942,  1.9139, -0.7609,  1.7101,  1.8918,  1.4691,  1.5987,
         1.1090, -0.4258,  0.9387,  0.2920,  1.2825,  2.5208,  1.8043,  1.9969,
         1.2420,  1.1446,  0.7551,  2.1238,  1.0311],
       grad_fn=<ReshapeAliasBackward0>)
tensor([-0.,  1.,  2., -1.,  2.,  2.,  1.,  2.,  1., -0.,  1.,  0.,  1.,  3.,
         2.,  2.,  1.,  1.,  1.,  2.,  1.], grad_fn=<RoundBackward0>)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([ 0.0081, -0.6171, -1.0463, -0.4368,  0.1799, -1.0544, -0.4031, -1.8607,
        -1.6785, -0.5161,  1.1321,  0.0252, -0.8810, -0.6366, -0.7090, -0.2673,
        -1.1872, -0.3035, -0.7169, -0.1758, -0.0563],
       grad_fn=<ReshapeAliasBackward0>)
tensor([ 0., -1., -1., -0.,  0., -1., -0., -2., -2., -1.,  1.,  0., -1., -1.,
        -1., -0., -1., -0., -1., -0., -0.], grad_fn=<RoundBac

KeyboardInterrupt: 